In [ ]:
from haystack import Pipeline
from haystack.components.builders.prompt_builder import PromptBuilder
from haystack.components.retrievers.in_memory import InMemoryBM25Retriever
from haystack_integrations.components.generators.ollama import OllamaGenerator
from haystack.components.converters import TextFileToDocument
from haystack.components.converters import PyPDFToDocument
from haystack.document_stores.in_memory import InMemoryDocumentStore
from pathlib import Path

converterText = TextFileToDocument()
converterPDF = PyPDFToDocument()

# Meta Parameters

In [ ]:
if not 'basic_concepts' in locals():
    basic_concepts = ""

if not 'process_text' in locals():
    process_text = ""

if not 'process_name' in locals():
    process_name = ""

if not 'large_language_model' in locals():
    large_language_model = ""

if not 'prompt_name' in locals():
    prompt_name = ""

if not 'loop_limit' in locals():
    loop_limit = "20"

if not 'llm_url' in locals():
    llm_url = "http://localhost:11434"

# print("process_name:", process_name)
# print("process description: ", process_text)
# print("llm:" , large_language_model)
# print("prompt_name:", prompt_name)

# DCR - in-context learning / RGA

In [ ]:
basicDCR = converterText.run(sources=[Path(basic_concepts)])

document_store = InMemoryDocumentStore()
document_store.write_documents(basicDCR['documents'])

# Prompt for Roles

In [ ]:
#Context
prompt_Role = """
[Context]
Imaging you are an expert in Business Process Management with extensive knowledge about declarative process modelling. Additionally, you have specific knowledge about the modelling technique "DCR" (Dynamic Condition Response), that you extend with information from the following documents:
{% for doc in documents %}
    {{ doc.content }}
{% endfor %}
I am going to feed you a textual description of some sort of Business Process. It could be a process in a company or organization. 
"""

#Task
prompt_Role = prompt_Role + """
{{ task }}
"""
task = """
Your task is to find all roles or actors for a DCR graph in the process description I will provide you with.
"""

# Output / Format
prompt_Role = prompt_Role + """
Return the roles in an array in JSON format, similar to: {"roles": [...,...,...]}. Do not forget the delimiter "," between the identified roles. Do not use any other delimiter. Do not add any other information to your answer. Do not include duplicate roles. Do not repeat the task description in your answer. Do neither describe nor explain your answer. 
"""

# Process Description
prompt_Role = prompt_Role + """
The process description in question is:
{{ process }}
"""

## Examples

In [ ]:
prompt_Role = prompt_Role + """

Consider the following as correct examples: 
First Example Description: "The process starts when the storage facility used by Puori sends a product sample to Ellipse. Ellipse then notifies Puori of the new sample. Before testing can begin, Puori then request a test of a sample through Ellipse. Ellipse must then test the product, as well as have additional testing done through Eurofarms."
Correctly identified roles: {"roles": ["Storage Facility", "Ellipse", "Puori"]}

Second Example Description: "When an employee creates an expense report the money must be paid out within one week. After creating the expense report a manager should approve the expense report. If a manager approves the  expense report the money can be paid out by finance. If the case is awaiting manager approval payout cannot happen."
Correctly identified roles: {"roles": ["Employee", "Manager", "Finance"]}

Consider the following as a wrong examples: 
Third Example Description: "The referee starts a new case regarding a certain doctor by interviewing patients.  Since a patient interview workflow is already  established, it is simply integrated in the new  workflow. Meanwhile, the director asks an external expert to review the work of the doctor under rating."
Wrongly identified roles: {"roles": ["Doctor", "Patient", "External Expert"]}
Why are they wrong?: receiving parties are not considered as roles; roles identified that should not be a role.
"""

## Combine Role Pipeline

In [ ]:
retriever = InMemoryBM25Retriever(document_store=document_store)
prompt_builder = PromptBuilder(template=prompt_Role)
llm = OllamaGenerator(model=large_language_model, url=llm_url, timeout=1200)

role_pipeline = Pipeline()
role_pipeline.add_component("retriever", retriever)
role_pipeline.add_component("prompt_builder", prompt_builder)
role_pipeline.add_component("llm", llm)
role_pipeline.connect("retriever", "prompt_builder.documents")
role_pipeline.connect("prompt_builder", "llm")

# Prompt for Activities

In [ ]:
#Context
prompt_Activity = """
Imaging you are an expert in Business Process Management with extensive knowledge about declarative process modelling. Additionally, you have specific knowledge about the modelling technique "DCR" (Dynamic Condition Response), that you extend with information from the following documents:
{% for doc in documents %}
    {{ doc.content }}
{% endfor %}
I am going to feed you a textual description of some sort of Business Process. It could be a process in a company or organization. 
"""

#Task
prompt_Activity = prompt_Activity + """
Your task is to find all activities or events for a DCR graph in the process description I will provide you with and map them to the following roles: {{ roles }}.
"""

# Output / Format
prompt_Activity = prompt_Activity + """
Return the activitites as keys in key-value-pairs in JSON format, with their associated role as value, which should look like this: { {"activity": ..., "role": ...}, ...}. Do not forget the delimiter "," between the identified activities. Do not use any other delimiter. Do not alter the text. Do not add any other information to your answer. Do not include duplicate activities. Do not repeat the task description in your answer. Do neither describe nor explain your answer. 
"""

# Process Description
prompt_Activity = prompt_Activity + """
The process description in question is:
{{ process }}
"""

## Examples

In [ ]:
prompt_Activity = prompt_Activity + """

Consider the following as correct examples: 
First Example Description: "Two weeks before the renting period is done, management needs to contact the seller to announce that the renting period is almost complete and ask the seller if they want an extension. However, it is also possible for sellers to contact management which then would not require management to contact them. If they do not want to extend, they will move out. If they would like an extension, management needs to check their revenue."
Correctly identified activities: [{"activity": "contact the seller to announce that the renting period is almost complete","role": "Management"},{"activity": "ask the seller if they want an extension","role": "Management"},{"activity": "contact management","role": "Seller"},{"activity": "do not want to extend","role": "Seller"},{"activity": "move out","role": "Seller"},{"activity": "like an extension","role": "Seller"},{"activity": "check their revenue","role": "Management"}]

Second Example Description: "A customer brings in a defective computer and the CRS checks the defect and hands out a repair cost calculation back. If the customer decides that the costs are acceptable, the process continues, otherwise she  takes her computer home unrepaired. The ongoing repair consists of two activities, which are executed, in an arbitrary order. The first activity is to check and repair the hardware, whereas the second activity checks and  configures the software."
Correctly dentified activities: [{"activity": "brings in a defective computer","role": "Customer"},{"activity": "checks the defect","role": "CRS"},{"activity": "hands out a repair cost calculation back","role": "CRS"},{"activity": "decides that the costs are acceptable","role": "Customer"},{"activity": "the process continues","role": "none"},{"activity": "takes her computer home unrepaired","role": "Customer"},{"activity": "check the hardware","role": "CRS"},{"activity": "repair the hardware","role": "CRS"},{"activity": "checks the software","role": "CRS"},{"activity": "configures the software","role": "CRS"}]

Consider the following as a wrong examples: 
Third Example Description: "If the seller does not confirm the shelf, the employee will pick another potential seller. After the confirmation the seller must decide their renting period which should be at least 14 weeks. Afterwards the seller needs to decide whether they pay over the phone or at the shop which requires an employee to take payment or to pay through the website which  automatically will take the payment."
Wrongly identified activities: [{"activity": "is picked by employee","role": "Seller"},{"activity": "should be at least 14 weeks","role": "Renting Period"},{"activity": "needs to decide","role": "Seller"}{"activity": "pay over the phone or at the shop", "role": "Seller"}{"activity": "take payment or pay through the website","role": "employee"}]
Why are they wrong?: activities have been cut off and split; activities have been combined; activities are wrongfully assigned to roles; activities are identified that should not be considered as activities
"""

## Combine Activity Pipeline

In [ ]:
retriever_Activity = InMemoryBM25Retriever(document_store=document_store)
prompt_builder_Activity = PromptBuilder(template=prompt_Activity)
llm2 = OllamaGenerator(model=large_language_model, url=llm_url, timeout=1200)

activity_pipeline = Pipeline()
activity_pipeline.add_component("retriever_Activity", retriever_Activity)
activity_pipeline.add_component("prompt_builder_Activity", prompt_builder_Activity)
activity_pipeline.add_component("llm2", llm2)
activity_pipeline.connect("retriever_Activity", "prompt_builder_Activity.documents")
activity_pipeline.connect("prompt_builder_Activity", "llm2")

# Prompt for Rules

In [ ]:
#Context
prompt_Rules = """
Imaging you are an expert in Business Process Management with extensive knowledge about declarative process modelling. Additionally, you have specific knowledge about the modelling technique "DCR" (Dynamic Condition Response), that you extend with information from the following documents:
{% for doc in documents %}
    {{ doc.content }}
{% endfor %}
I am going to feed you a textual description of some sort of Business Process. It could be a process in a company or organization. 
"""

#Task
prompt_Rules = prompt_Rules + """
Your task is to find and define all constraints between the activities for a DCR graph in the process description I will provide you with, based on the already identified activities: {{ activities }}.
A constraint must be in accordance with DCR a Condition, Response, Exclude, Include or Milestone, and its associated activities are a source activity and a target activity.
"""

# Output / Format
prompt_Rules = prompt_Rules + """
Return the DCR constraints as keys in key-value-pairs in JSON format, which should look like this: { {"constraint": ..., "source activity: ..., "target activity": ...}, ...}. The constraint should be one of the constraint types of DCR and the activities its associated source and target activity. Do not forget the delimiter "," between the identified constraints. Do not use any other delimiter. Do not alter the text. Do not add any other information to your answer. Do not include duplicate constraints. Do not include the roles of the activities in the output. Do not repeat the task description in your answer. Do neither describe nor explain your answer. 
"""

# Process Description
prompt_Rules = prompt_Rules + """
The process description in question is:
{{ process }}
"""

## Examples

In [ ]:
prompt_Rules = prompt_Rules + """

Consider the following examples. The first two for each constraint type are correct examples, the third one is a wrong example for the respective constraint type:

First Example for Condition: "Once they have completed everything, they can get the bike from the shop and the contract will be started."
Correctly identified Constraint: [{"constraint": "Condition", "source activity": "have completed everything", "target activity": "get the bike from the shop"},{"constraint": "Condition", "source activity": "have completed everything", "target activity": "the contract will be started"}]

Second Example for Condition: "Afterwards the GRM must request more information from the potential retailer. The retailer can only provide the requested information after the request has been made."
Correctly identified Constraint: {"constraint": "Condition", "source activity": "request more information from the potential retailer", "target activity": "provide the requested information"}

Third Example for Condition: "The GRM receives a request to become retailer. Afterwards the GRM must request more information from the potential retailer."
Wrongly identified Constraint: {"constraint": "Condition", "source activity": "receives a request to become retailer", "target activity": "request more information from the potential retailer"}
Why is it wrong?: The sentences state an obligation for the target activity, but not a conditional permission given by performing the source activity.

First Example for Response: "If they do not want to extend, they will move out."
Correctly identified Constraint: {"constraint": "Response", "source activity": "do not want to extend", "target activity": "move out"}

Second Example for Response: "If no bikes are available, Swapfiets online customer service will notify the customer, and the service will not be supported."
Correctly identified Constraint: [{"constraint": "Response", "source activity": "no bikes are available", "target activity": "notify the customer"},{"constraint": "Response", "source activity": "no bikes are available", "target activity": "the service is not supported"}]

Third Example for Response: "If the revenue is equal or above 700 DKK a week then they can extend the renting period."
Wrongly identified Constraint: {"constraint": "Response", "source activity": "is equal or above 700 DKK a week", "target activity": "extend the renting period"}
Why is it wrong?: The sentence states a conditional permission for the target activity, but not an obligation.

First Example for Include: "A customer must browse for products prior to adding a product to the basket, but they can access the basket upon entering the webshop without any prior conditions. In order to proceed to check-out at least one product must be in the basket."
Correctly identified Constraint: {"constraint": "Include", "source activity": "adding a product to the basket", "target activity": "proceed to check-out"}

Second Example for Include: "Only once the contract has been created the Head of Development can send the contract."
Correctly identified Constraint: {"constraint": "Include", "source activity": "create the contract", "target activity": "send the contract"}

Third Example for Include: "If a manager approves the expense report the money can be paid out by finance."
Wrongly identified Constraint: {"constraint": "Include", "source activity": "approves the expense report", "target activity": "pay out the money"}
Why is it wrong?: The sentence does not state a strict consequence that would otherwise declare the target activity to be excluded from the process.

First Example for Exclude: "Two weeks before the renting period is done, management needs to contact the seller to announce that the renting period is almost complete and ask the seller if they want an extension. However, it is also possible for sellers to contact management which then would not require management to contact them."
Correctly identified Constraint: {"constraint": "Exclude", "source activity": "contact management", "target activity": "contact the seller to announce that the renting period is almost complete"}

Second Example for Exclude: "If an error is detected another arbitrary repair activity is executed, otherwise the repair is finished."
Correctly identified Constraint: [{"constraint": "Exclude", "source activity": "execute another arbitrary repair activity", "target activity": "the repair is finished"},{"constraint": "Exclude", "source activity": "the repair is finished", "target activity": "execute another arbitrary repair activity"}]

Third Example for Exclude: "If errors are found, Clean Label must correct the errors before publishing the results again."
Wrongly identified Constraint: {"constraint": "Exclude", "source activity": "errors are found", "target activity": "publishing the results again"}
Why is it wrong?: The sentence does not strictly state whether the target activity is excluded from the process if the source activity is performed.

First Example for Milestone: "After the design is approved by the product owner, Marketing may send the design to test groups for feedback. After the design is approved, if the design is not sent to test groups, Marketing must send the design to P&R."
Correctly identified Constraint: {"constraint": "Milestone", "source activity": "approve the design", "target activity": "send the design to P&R"}

Second Example for Milestone: If the case is awaiting manager approval payout cannot happen. If the manager rejects the expense report he must later approve the expense report in order to payout to happen.
Correctly identified Constraint: {"constraint": "Milestone", "source activity": "approve the expense report", "target activity": "publishing the results again"}

Third Example for Milestone: "If errors are found, Clean Label must correct the errors before publishing the results again."
Wrongly identified Constraint: {"constraint": "Milestone", "source activity": "correct the errors", "target activity": "publishing the results again"}
Why is it wrong?: The sentence does not invoke any conditions on the source activity and, thus, it cannot be a milestone for the target activity.
"""

### Combine Rule Pipeline

In [ ]:
retriever_Rules = InMemoryBM25Retriever(document_store=document_store)
prompt_builder_Rules = PromptBuilder(template=prompt_Rules)
llm3 = OllamaGenerator(model=large_language_model, url=llm_url, timeout=1200)

rule_pipeline = Pipeline()
rule_pipeline.add_component("retriever_Rules", retriever_Rules)
rule_pipeline.add_component("prompt_builder_Rules", prompt_builder_Rules)
rule_pipeline.add_component("llm3", llm3)
rule_pipeline.connect("retriever_Rules", "prompt_builder_Rules.documents")
rule_pipeline.connect("prompt_builder_Rules", "llm3")

# Graph Generation

In [ ]:
import json
import xml.etree.ElementTree as ET 
import uuid
from pathlib import Path
from datetime import datetime

## Run Prompt Pipeline

In [ ]:
def runPromptPipeline(): 
    start_time_role = datetime.now()
    response_Roles = role_pipeline.run(
        {
            "retriever": {"query": prompt_Role},
            "prompt_builder": {"task": task, "process": process_text},
        }
    )
    processing_time_role = datetime.now() - start_time_role
    processing_duration_role = processing_time_role.total_seconds()
    input_tokens_Role = response_Roles["llm"]["meta"][0]["prompt_eval_count"]
    output_tokens_Role = response_Roles["llm"]["meta"][0]["eval_count"]
    cleaned_roles = ''.join(response_Roles["llm"]["replies"])
    cleaned_roles = cleaned_roles.replace("'", '').replace("{", '').replace("}", '').replace("[", '').replace("]", '').replace(":", '').replace("\"roles\"", '').replace("\n",'').replace(" ", '').replace("\",\"",'\", \"').replace("\\",'')

    start_time_activity = datetime.now()
    response_Activities = activity_pipeline.run(
        {
            "retriever_Activity": {"query": prompt_Activity},
            "prompt_builder_Activity": {"process": process_text, "roles": cleaned_roles},
        }
    )
    processing_time_activity = datetime.now() - start_time_activity
    processing_duration_activity = processing_time_activity.total_seconds()
    input_tokens_Activity = response_Activities["llm2"]["meta"][0]["prompt_eval_count"]
    output_tokens_Activity = response_Activities["llm2"]["meta"][0]["eval_count"]
    cleaned_activities = ''.join(response_Activities["llm2"]["replies"])
    cleaned_activities = cleaned_activities.replace("'", '').replace("[", '').replace("]", '').replace("\"roles\"", '').replace("\n",'').replace("\",\"",'\", \"').replace("\\",'').replace('}{"activity','},{"activity').replace('} {"activity','},{"activity')

    start_time_rule = datetime.now()
    response_Rules = rule_pipeline.run(
        {
            "retriever_Rules": {"query": prompt_Rules},
            "prompt_builder_Rules": {"process": process_text, "activities": cleaned_activities},
        }
    )
    processing_time_rule = datetime.now() - start_time_rule
    processing_duration_rule = processing_time_rule.total_seconds()
    input_tokens_Rule = response_Rules["llm3"]["meta"][0]["prompt_eval_count"]
    output_tokens_Rule = response_Rules["llm3"]["meta"][0]["eval_count"]
    cleaned_rules = ''.join(response_Rules["llm3"]["replies"])
    cleaned_rules = cleaned_rules.replace("'", '').replace("[", '').replace("]", '').replace("\n",'').replace("  ",'').replace("\",\"",'\", \"').replace("\\",'').replace('}{"constraint','},{"constraint').replace('} {"constraint','},{"constraint')

    return cleaned_roles, processing_duration_role, input_tokens_Role, output_tokens_Role, cleaned_activities, processing_duration_activity, input_tokens_Activity, output_tokens_Activity, cleaned_rules, processing_duration_rule, input_tokens_Rule, output_tokens_Rule


## Create JSON from Output

In [ ]:
def generateJSON(roles,activities,rules):
    rolesString = '"roles": [' + roles + ']'
    activitiesString = '"activities": [' + activities + ']'
    rulesString = '"constraints": [' + rules + ']'
    graphString = '{"DCRgraph": {' + rolesString + ', ' + activitiesString + ', ' + rulesString + '} }'
    
    # print(rolesString)
    # print(activitiesString)
    # print(rulesString)
    # print(graphString)

    graphJSON = json.loads(graphString)

    return graphJSON

## Generate DCR Editor XML

In [ ]:
### the code for json_to_xml() was mostly generated using GPT-4o mini
def json_to_xml(json_obj):
    # Create the root element
    definitions = ET.Element("dcr:definitions", {
        "xmlns:dcr": "http://tk/schema/dcr",
        "xmlns:dcrDi": "http://tk/schema/dcrDi",
        "xmlns:dc": "http://www.omg.org/spec/DD/20100524/DC"
    })

    # Create dcrGraph element
    dcr_graph = ET.SubElement(definitions, "dcr:dcrGraph", id="dcrGraph")

    # Create events
    event_ids = {}
    for activity in json_obj["DCRgraph"]["activities"]:
        role = activity["role"]
        description = activity["activity"]
        event_id = f"Event_{uuid.uuid4().hex[:8]}"  # Generate a unique event ID
        event_ids[event_id] = activity  # Store the event ID with its activity for later use

        event = ET.SubElement(dcr_graph, "dcr:event", {
            "id": event_id,
            "role": role,
            "description": description,
            "included": "true",
            "executed": "false",
            "pending": "false",
            "enabled": "false"
        })

    # Create relations
    relation_ids = []
    for constraint in json_obj["DCRgraph"]["constraints"]:
        source_activity = constraint["source activity"]
        target_activity = constraint["target activity"]
        relation_id = f"Relation_{uuid.uuid4().hex[:8]}"  # Generate a unique relation ID

        #check whether constraint type matches DCR Editor XML types
        if constraint["constraint"] == "Condition":
            constraint_type = "condition"
        elif constraint["constraint"] == "Response":
            constraint_type = "response"
        elif constraint["constraint"] == "Exclude":
            constraint_type = "exclude"
        elif constraint["constraint"] == "Include":
            constraint_type = "include"
        elif constraint["constraint"] == "Milestone":
            constraint_type = "milestone"
        else:
            raise TypeError("Constraint Type was not identified correctly")
        
        # Find the corresponding event IDs
        source_event_id = next((eid for eid, act in event_ids.items() if act["activity"] == source_activity), None)
        target_event_id = next((eid for eid, act in event_ids.items() if act["activity"] == target_activity), None)

        if source_event_id and target_event_id:
            # Create the dcr:relation with a unique ID
            ET.SubElement(dcr_graph, "dcr:relation", {
                "id": relation_id,
                "type": constraint_type, 
                "sourceRef": source_event_id,
                "targetRef": target_event_id
            })
            relation_ids.append(relation_id)  # Store the relation ID for later use

    # Create dcrRootBoard element
    root_board = ET.SubElement(definitions, "dcrDi:dcrRootBoard", id="RootBoard")
    plane = ET.SubElement(root_board, "dcrDi:dcrPlane", id="Plane", boardElement="dcrGraph")

    # Create shapes for the diagram
    for event_id in event_ids.keys():
        shape = ET.SubElement(plane, "dcrDi:dcrShape", id=f"{event_id}_di", boardElement=event_id)
        # Example bounds, you can adjust the coordinates as needed
        ET.SubElement(shape, "dc:Bounds", x="640", y="230", width="130", height="150")

    # Create dcrDi:relation elements with unique IDs
    for relation_id in relation_ids:
        # Create a corresponding dcrDi:relation
        relation_di_id = f"{relation_id}_di"  # ID for dcrDi:relation
        relation_di = ET.SubElement(plane, "dcrDi:relation", id=relation_di_id, boardElement=relation_id)
        # Example waypoints, you can adjust the coordinates as needed
        ET.SubElement(relation_di, "dcrDi:waypoint", x="705", y="380")
        ET.SubElement(relation_di, "dcrDi:waypoint", x="705", y="400")
        ET.SubElement(relation_di, "dcrDi:waypoint", x="620", y="400")
        ET.SubElement(relation_di, "dcrDi:waypoint", x="620", y="305")
        ET.SubElement(relation_di, "dcrDi:waypoint", x="640", y="305")

    # Convert to string
    xml_str = ET.tostring(definitions, encoding='UTF-8', xml_declaration=True).decode('UTF-8')
    return xml_str



## Combined Graph Generation

In [ ]:
current_run = 0
generation_start_time = datetime.now()
errors = 0
role_times = []
role_input_token = []
role_output_token = []
activity_times = []
activity_input_token = []
activity_output_token = []
rule_times = []
rule_input_token = []
rule_output_token = []

while True:
    if current_run >= loop_limit:
        # safety mechanism to ensure termination if an unexpected issue is not handled
        # important to leave in when running everything automatically
        raise ValueError("Exceeded maximumg number of runs. Generation of Graph failed.")
    try:
        roles, role_time, role_input, role_output, activities, activity_time, activity_input, activity_output, rules, rule_time, rule_input, rule_output = runPromptPipeline()
        role_times.append(role_time)
        role_input_token.append(role_input)
        role_output_token.append(role_output)
        activity_times.append(activity_time)
        activity_input_token.append(activity_input)
        activity_output_token.append(activity_output)
        rule_times.append(rule_time)
        rule_input_token.append(rule_input)
        rule_output_token.append(rule_output)
        graphJSON = generateJSON(roles,activities,rules)
        print("Parsing to JSON successful")
        xml = json_to_xml(graphJSON)
        print(f"XML Generation successful ({process_name},Pipeline_A1_{prompt_name})")
        print("xml:", xml)
        print("\n")
        break
    except Exception as e:
        current_run += 1
        errors += 1
        print(f"Graph Generation failed with error ({process_name},Pipeline_A1_{prompt_name})")
        print(e, flush=True)
        runs_left = loop_limit - current_run
        print("Iterations left: " + str(runs_left))
        print("\n")

generation_finished_time = datetime.now() - generation_start_time
generation_duration = generation_finished_time.total_seconds()

## Save Time to Results

In [ ]:
new_times = f"{process_name};{generation_duration};{role_times};{role_input_token};{role_output_token};{activity_times};{activity_input_token};{activity_output_token};{rule_times};{rule_input_token};{rule_output_token};{errors}\n"

folder = Path('./6_Time_Analysis') / "data" / large_language_model / "Pipeline_A1"
folder.mkdir(parents=True, exist_ok=True) 

file_name = f"{prompt_name}.txt"
file_path = folder / file_name 

with open(file_path, 'a+') as file:
    file.write(new_times)

## Save XML to Results

In [ ]:
folder = Path(result_xml_path) / large_language_model / "Pipeline_A1" / prompt_name
folder.mkdir(parents=True, exist_ok=True) 

file_name = f"{process_name}_{large_language_model}_{prompt_name}_{datetime.now().date()}_{datetime.now().time()}.xml"
file_path = folder / file_name 

with open(file_path, 'w') as result_destination:
    result_destination.write(xml)